In [1]:
# pip install frozendict 
# pip install pyarrow
# pip install -e ./streamline_package
# pip install ray[default]

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
from streamline.delayed import delay_lib, step, Delayed, eval_delay
from streamline import RunEnv, load_runenv, Pipeline, Function, Var
from streamline.utilities import mainify

In [4]:
from typing import *
import pandas as pd

In [5]:
import ray

ray.init()

2025-03-17 21:57:07,479	INFO worker.py:1832 -- Started a local Ray instance. View the dashboard at http://127.0.0.1:8266 


Python version:,3.11.6
Ray version:,2.43.0
Dashboard:,http://127.0.0.1:8266


(<lambda> pid=26144) 2061858284512
(<lambda> pid=42588) 2875524555744
(<lambda> pid=26144) 2061858284512
(<lambda> pid=42588) 2875524555744
(<lambda> pid=23860) 2143827895264
(<lambda> pid=27616) 2079607792608
(<lambda> pid=39140) 2070655706080
(<lambda> pid=184) 2063738250208
(<lambda> pid=26836) 2338110940128
(<lambda> pid=41928) 2484224500704
(<lambda> pid=33896) 3010551063520
(<lambda> pid=24712) 2863580292064


In [6]:
renv = RunEnv({
    'a': 3,
    'df': pd.DataFrame([[4]])
})

renv.save(
    './ex/'
)
# display(renv._env)
renv2 = load_runenv('./ex/')

In [7]:
import numpy as np
import time

def dummy():
    print(id(time))
    time.sleep(1)
    return np.random.rand()
    
def dummy2(env, kw):
    env['myid'] = id(env)
    env['lst'].append(id(env))

pipe = Pipeline([
    Function(lambda : 3, arg_cat='s')
]).add_step(
    step=Function(
        fun=lambda x, s, e, f, h: (x**2+1, s, e, f, h),
        args=Var('a'),
        kw={'e': Var('a'), 'f': 8, 'h': 5},
        out_var=('b', 'c', 'e', 'f', 'h'),
        arg_cat='f',
    )
).add_import_lib(
    {'numpy': 'np', 'pandas': 'pd'},
    index=0
).add_step(Function(dummy, out_var='r'))
pipe
env = pipe.run(RunEnv({'a': 3, 'e': 'k', 'lst': []}), {'f_s': 'f_d', 'f_e': 'f_n', 'f_h': 3})
env._env

1515248655776


{'a': 3,
 'e': 'f_n',
 'lst': [],
 'np': <module 'numpy' from 'C:\\Users\\Nagato\\AppData\\Local\\Programs\\Python\\Python311\\Lib\\site-packages\\numpy\\__init__.py'>,
 'pd': <module 'pandas' from 'C:\\Users\\Nagato\\AppData\\Local\\Programs\\Python\\Python311\\Lib\\site-packages\\pandas\\__init__.py'>,
 '__preamble__': {'__steps__': [{'step': Function(arg_cat='', tags={'import_lib', 'function'}),
    'kwargs': {}},
   {'step': Function(arg_cat='s', tags={'function'}), 'kwargs': {}},
   {'step': Function(arg_cat='f', tags={'function'}),
    'kwargs': {'s': 'f_d', 'e': 'f_n', 'h': 3}},
   {'step': Function(arg_cat='', tags={'function'}), 'kwargs': {}}]},
 '_': 3,
 'b': 10,
 'c': 'f_d',
 'f': 8,
 'h': 3,
 'r': 0.18563471004316667}

In [8]:
pipe.run(RunEnv({'a': 3, 'e': 'k', 'lst': []}), {'f_s': 'f_d', 'f_e': 'f_n', 'f_h': 3})

1515248655776


{'a': 3, 'e': 'f_n', 'lst': [], 'np': <module 'numpy' from 'C:\\Users\\Nagato\\AppData\\Local\\Programs\\Python\\Python311\\Lib\\site-packages\\numpy\\__init__.py'>, 'pd': <module 'pandas' from 'C:\\Users\\Nagato\\AppData\\Local\\Programs\\Python\\Python311\\Lib\\site-packages\\pandas\\__init__.py'>, '__preamble__': {'__steps__': [{'step': Function(arg_cat='', tags={'import_lib', 'function'}), 'kwargs': {}}, {'step': Function(arg_cat='s', tags={'function'}), 'kwargs': {}}, {'step': Function(arg_cat='f', tags={'function'}), 'kwargs': {'s': 'f_d', 'e': 'f_n', 'h': 3}}, {'step': Function(arg_cat='', tags={'function'}), 'kwargs': {}}]}, '_': 3, 'b': 10, 'c': 'f_d', 'f': 8, 'h': 3, 'r': 0.8691561070952463}

In [9]:
from streamline.pipeline import run_parallel

res_par = run_parallel([
    (pipe, {'a': 3, 'e': 'k', 'lst': []}, {'f_s': 'f_d', 'f_e': 'f_n', 'f_h': 3}),
    (pipe, {'a': 3, 'e': 'k', 'lst': []}, {'f_s': 'f_d', 'f_e': 'f_n', 'f_h': 3}),
])

In [10]:
res_par[1]

{'a': 3, 'e': 'f_n', 'lst': [], 'np': <module 'numpy' from 'C:\\Users\\Nagato\\AppData\\Local\\Programs\\Python\\Python311\\Lib\\site-packages\\numpy\\__init__.py'>, 'pd': <module 'pandas' from 'C:\\Users\\Nagato\\AppData\\Local\\Programs\\Python\\Python311\\Lib\\site-packages\\pandas\\__init__.py'>, '__preamble__': {'__steps__': [{'step': Function(arg_cat='', tags={'import_lib', 'function'}), 'kwargs': {}}, {'step': Function(arg_cat='s', tags={'function'}), 'kwargs': {}}, {'step': Function(arg_cat='f', tags={'function'}), 'kwargs': {'s': 'f_d', 'e': 'f_n', 'h': 3}}, {'step': Function(arg_cat='', tags={'function'}), 'kwargs': {}}]}, '_': 3, 'b': 10, 'c': 'f_d', 'f': 8, 'h': 3, 'r': 0.5312587045812112}

In [11]:
from streamline import eval_delay, step

eval_delay(step.arg_cat, {'step': Function(lambda x: x, arg_cat='s')})

's'

In [12]:
base_env = RunEnv({'a': 3, 'e': 'k', 'lst': []})
env_l = pipe.run_parallel([base_env], [{'f_s': 'f_d', 'f_e': 'f_n', 'f_h': 3} for i in range(10)])
env_l

[[{'a': 3, 'e': 'f_n', 'lst': [], 'np': <module 'numpy' from 'C:\\Users\\Nagato\\AppData\\Local\\Programs\\Python\\Python311\\Lib\\site-packages\\numpy\\__init__.py'>, 'pd': <module 'pandas' from 'C:\\Users\\Nagato\\AppData\\Local\\Programs\\Python\\Python311\\Lib\\site-packages\\pandas\\__init__.py'>, '__preamble__': {'__steps__': [{'step': Function(arg_cat='', tags={'import_lib', 'function'}), 'kwargs': {}}, {'step': Function(arg_cat='s', tags={'function'}), 'kwargs': {}}, {'step': Function(arg_cat='f', tags={'function'}), 'kwargs': {'s': 'f_d', 'e': 'f_n', 'h': 3}}, {'step': Function(arg_cat='', tags={'function'}), 'kwargs': {}}]}, '_': 3, 'b': 10, 'c': 'f_d', 'f': 8, 'h': 3, 'r': 0.132491865984531},
  {'a': 3, 'e': 'f_n', 'lst': [], 'np': <module 'numpy' from 'C:\\Users\\Nagato\\AppData\\Local\\Programs\\Python\\Python311\\Lib\\site-packages\\numpy\\__init__.py'>, 'pd': <module 'pandas' from 'C:\\Users\\Nagato\\AppData\\Local\\Programs\\Python\\Python311\\Lib\\site-packages\\pandas

In [13]:
base_env['lst']

[]

In [14]:
[c['lst'] for c in env_l[0]]

[[], [], [], [], [], [], [], [], [], []]

In [15]:
s = list(range(10))
s = [s[i:i+5] for i in range(0, 10, 5)]
s

[[0, 1, 2, 3, 4], [5, 6, 7, 8, 9]]

In [16]:
env.gen_pipeline()

Pipeline(steps=[
  Function(arg_cat='', tags={'import_lib', 'function'}), 
  Function(arg_cat='s', tags={'function'}), 
  Function(arg_cat='f', tags={'function'}), 
  Function(arg_cat='', tags={'function'})
], )

In [17]:
pipe

Pipeline(steps=[
  Function(arg_cat='', tags={'import_lib', 'function'}), 
  Function(arg_cat='s', tags={'function'}), 
  Function(arg_cat='f', tags={'function'}), 
  Function(arg_cat='', tags={'function'})
], )

In [18]:
pipe.loc[[0, 1]]

Pipeline(steps=[
  Function(arg_cat='', tags={'import_lib', 'function'}), 
  Function(arg_cat='s', tags={'function'})
], )

In [19]:
pipe.loc[[0, 1]] = Function(arg_cat='d', fun=lambda x: x)

In [20]:
pipe

Pipeline(steps=[
  Function(arg_cat='d', tags={'function'}), 
  Function(arg_cat='d', tags={'function'}), 
  Function(arg_cat='f', tags={'function'}), 
  Function(arg_cat='', tags={'function'})
], )

In [21]:
step.arg_cat.startswith('s')

step.arg_cat.startswith(*('s',), **{})

In [22]:
pipe.loc[step.arg_cat.startswith('d')]

Pipeline(steps=[
  Function(arg_cat='d', tags={'function'}), 
  Function(arg_cat='d', tags={'function'})
], )

In [23]:
pipe.loc[pipe.apply(lambda x: x.tags is None)]

Pipeline(steps=[
], )

In [24]:
pipe.to_list()

[Function(arg_cat='d', tags={'function'}),
 Function(arg_cat='d', tags={'function'}),
 Function(arg_cat='f', tags={'function'}),
 Function(arg_cat='', tags={'function'})]

In [25]:
step.tags.contains(3)

(3 in step.tags)

In [26]:
step = Delayed(prefix='step')
d = delay_lib.type((step.tags.contains(3)) & (step.tags.contains(1))).__name__.isin(['bool'])
print(d)
eval_delay(d, env={'step': Function(arg_cat='s', fun=lambda x: x, tags={1, 2})})

(type(*(((3 in step.tags) & (1 in step.tags)),), **{}).__name__ in ['bool'])


True

In [27]:
eval_delay(step.tags.contains(3), env={'step': Function(arg_cat='s', fun=lambda x: x, tags={1, 2})})

False

In [28]:
import dis

a = 3
def s():
    return a

def get_global_dependencies(func):
    """
    Returns a set of global variable names accessed by the function.
    """
    dependencies = set()
    for instr in dis.get_instructions(func):
        if instr.opname == 'LOAD_GLOBAL':
            dependencies.add(instr.argval)
    return dependencies

get_global_dependencies(Pipeline.run)

{'dict', 'sl', 'type'}